In [ ]:
import numpy as np
from osim.env import ProstheticsEnv
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
import torch
import torch.nn as nn
import pickle

In [ ]:
nproc = 4
dataset=dict()

In [ ]:
def make_env(seed):
    def _f():
        env = ProstheticsEnv(visualize=False)
        env.change_model(seed=seed)
        return env
    return _f

In [ ]:
envs = [make_env(seed) for seed in range(nproc)]
envs = SubprocVecEnv(envs)
obs = envs.reset()

In [ ]:
D_in=envs.observation_space.shape[0]
D_out=envs.action_space.shape[0]
H=512
device = torch.device('cpu')

In [ ]:
random_policy = torch.nn.Sequential(
                torch.nn.Linear(D_in, H),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H),
                torch.nn.ReLU(),
                torch.nn.Linear(H, D_out),
                ).to(device)

In [ ]:
done=[False]
i=0
while not (any(done)):
# for i in range(10):
    action=random_policy(torch.from_numpy(obs).float()).data.numpy()
    obs_next,_,done,_=envs.step(action)
    dataset[i]=[obs,action,obs_next]
    obs=obs_next
    i+=1

In [ ]:
dataset.keys()

In [ ]:
pickle.dump(dataset,open('data_haris.pkl','wb'))